In [ ]:
# install packages
!pip install datasets evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.8 MB/s eta 0:00:00


In [ ]:
!pip install transformers==4.28.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 38.0 MB/s eta 0:00:00


In [ ]:
# Build the audio model
import tensorflow as tf
from transformers import TFAutoModel
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense

# Load the pre-trained model using the Hugging Face interface
model_checkpoint = "facebook/wav2vec2-base"
huggingface_model = TFAutoModel.from_pretrained(model_checkpoint, trainable=False, from_pt=True)

# Define the inputs to the model
input_values = tf.keras.Input(shape=(16000,), dtype=tf.float32)

# Pass the inputs through the Wav2Vec model
wav2vec_output = huggingface_model(input_values)

# Reshape word_model output to match the shape of audio_model output
audio_model_output = layers.GlobalAveragePooling1D()(wav2vec_output[1])
# Drop-out layer before the final Classification-Head
audio_model_output = layers.Dropout(0.5) (audio_model_output)

output = Dense(1, activation='sigmoid')(audio_model_output)

# Create the TensorFlow functional API model
audio_model = tf.keras.Model(inputs=input_values, outputs=output)

audio_model.summary()

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:380: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(

TFWav2Vec2Model has backpropagation operations that are NOT supported on CPU. If you wish to train/fine-tune this model, you need a GPU or a TPU
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFWav2Vec2Model: ['quantizer.weight_proj.weight', 'quantizer.codevectors', 'quantizer.weight_proj.bias', 'project_hid.weight', 'project_q.bias', 'project_hid.bias', 'project_q.weight']
- This IS expected if you are initializing TFWav2Vec2Model from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a Bert

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 16000)]           0         
                                                                 
 tf_wav2_vec2_model_1 (TFWav  TFWav2Vec2BaseModelOutpu  94371712 
 2Vec2Model)                 t(last_hidden_state=(Non            
                             e, 49, 768),                        
                              extract_features=(None,            
                              49, 512),                          
                              hidden_states=None, att            
                             entions=None)                       
                                                                 
 global_average_pooling1d_1   (None, 512)              0         
 (GlobalAveragePooling1D)                                        
                                                           

In [ ]:
# Train the audio model
audio_model.compile(loss='binary_crossentropy', optimizer='adam',  metrics=['accuracy', tf.keras.metrics.Precision(),tf.keras.metrics.Recall(), tf.keras.metrics.AUC()])

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                            patience=10,
                                            restore_best_weights=True)
audio_model.fit(audio_train, y_train,
                       epochs=50, batch_size=16, validation_data=(audio_val, y_val),
                       callbacks=callback)

Epoch 1/50
378/378 [==============================] - 62s 140ms/step - loss: 0.6823 - accuracy: 0.5757 - precision_1: 0.4418 - recall_1: 0.1686 - auc_1: 0.5376 - val_loss: 0.6825 - val_accuracy: 0.5767 - val_precision_1: 0.5595 - val_recall_1: 0.0723 - val_auc_1: 0.5602
Epoch 2/50
378/378 [==============================] - 47s 125ms/step - loss: 0.6702 - accuracy: 0.5884 - precision_1: 0.4830 - recall_1: 0.1849 - auc_1: 0.5712 - val_loss: 0.6855 - val_accuracy: 0.5787 - val_precision_1: 0.5714 - val_recall_1: 0.0800 - val_auc_1: 0.5775
Epoch 3/50
378/378 [==============================] - 48s 126ms/step - loss: 0.6683 - accuracy: 0.5974 - precision_1: 0.5109 - recall_1: 0.2105 - auc_1: 0.5789 - val_loss: 0.6755 - val_accuracy: 0.5741 - val_precision_1: 0.5405 - val_recall_1: 0.0615 - val_auc_1: 0.6038
Epoch 4/50
378/378 [==============================] - 48s 126ms/step - loss: 0.6640 - accuracy: 0.6041 - precision_1: 0.5304 - recall_1: 0.2239 - auc_1: 0.5933 - val_loss: 0.6669 - val_ac

In [ ]:
# Evaluate the audio model
audio_model.evaluate(audio_test, y_test)

60/60 [==============================] - 11s 178ms/step - loss: 0.6545 - accuracy: 0.5947 - precision_1: 0.5016 - recall_1: 0.2021 - auc_1: 0.6212


[0.6545339226722717,
 0.5947089791297913,
 0.5016181468963623,
 0.20208604633808136,
 0.6212417483329773]